In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import numpy as np
from datetime import datetime as dt

from ruins.core import build_config
from ruins.plotting import plot_extreme_pdf
config, dm = build_config()

### UNCERTAINTY

In [40]:
STEP = 4
# build some default data
x = np.linspace(0, 30, 200)
y1 = stats.norm.pdf(x, loc=12, scale=2)

fig = go.Figure()

fig.add_trace(go.Scatter(x=[12, 12], y=[0, stats.norm.pdf(12, loc=12, scale=2)], mode='lines', line=dict(color='purple', dash='dash', width=5), showlegend=False))

fig.update_layout(
    width=500, height=350,
    template='plotly_white',
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis=dict(range=(0,30), title='Time [min]'),
    #yaxis=dict(range=(0, 1))
)

if STEP > 1:
    fig.add_trace(go.Scatter(x=x, y=y1, mode='lines', line=dict(color='purple'), fill='tozerox', showlegend=False))

if STEP > 2:
    y2 = stats.norm.pdf(x, loc=22, scale=2.)
    fig.add_trace(go.Scatter(x=x, y=y2, mode='lines', line=dict(color='darkslateblue'), fill='tozerox', showlegend=False))
    fig.add_trace(go.Scatter(x=[22, 22], y=[0, stats.norm.pdf(22, loc=22, scale=2.)], mode='lines', line=dict(color='darkslateblue', dash='dash', width=5), showlegend=False))

if STEP > 3:
    y3 = stats.norm.pdf(x, loc=19, scale=3.7)
    fig.add_trace(go.Scatter(x=x, y=y3, mode='lines', line=dict(color='darkgreen'), fill='tozerox', showlegend=False))
    fig.add_trace(go.Scatter(x=[19, 19], y=[0, stats.norm.pdf(19, loc=19, scale=3.7)], mode='lines', line=dict(color='darkgreen', dash='dash', width=5), showlegend=False))


fig.write_image(f'uncertainty_step{STEP}.png')
fig

### Weather and Climate

In [45]:
STEP = 2


fig = make_subplots(rows=5, cols=1, specs=[[{}], [{'rowspan': 4}], [None], [None], [None]])
arr = dm['weather'].read()
tmax = arr.sel(vars='Tmax')

# base settings
fig.update_layout(
    width=600, height=500,
    template='plotly_white',
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    #xaxis=dict(range=(0,30), title='Time [min]'),
    #yaxis=dict(range=(0, 1))
)
if STEP == 0:
    x = tmax.get_index('time').to_pydatetime()
    for name in tmax.keys():
        fig.add_trace(
            go.Scatter(x=x, y=tmax[name].values, mode='lines', line=dict(color='grey', width=1.), showlegend=False),
            row=2, col=1
        )
    fig.update_layout(
        xaxis2_title='Time',
        yaxis2_title='max. Temperature'
    )
elif STEP < 4:
    tmax = tmax.sel(time=slice('1980-01-01', '1980-12-31'))
    x = tmax.get_index('time').to_pydatetime()
    for name in tmax.keys():
        fig.add_trace(
            go.Scatter(x=x, y=tmax[name].values, mode='lines', line=dict(color='grey', width=1.), showlegend=False),
            row=2, col=1
        )
    
    if STEP > 1:
        
        fig.add_trace(
            go.Scatter(x=x, y=tmax['Cuxhaven'].values, mode='lines', line=dict(color='yellow', width=0.5), showlegend=False),
            row=2, col=1
        )
    fig.update_layout(
        xaxis2_title='Time',
        yaxis2_title='max. Temperature'
    )

    if STEP > 2:
        data = tmax['Cuxhaven'].to_dataframe()['Cuxhaven']
        grp = data.groupby([dt.strftime(d, '%m') for d in data.index.to_pydatetime()]).max()

        fig.add_trace(
            go.Heatmap(z=[grp.values], colorscale='RdBu_r', showscale=False, showlegend=False),
            row=1, col=1
        )

        disable = dict(showgrid=False, zeroline=False, showline=False, ticks='', showticklabels=False)
        fig.update_layout(xaxis1=disable, yaxis1=disable)
else:
    fig = go.Figure()
    fig.update_layout(
        width=600, height=700,
        template='plotly_white',
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis_title='Month',
        yaxis_title='Year'
    )

    data = tmax['Cuxhaven'].to_dataframe()['Cuxhaven']
    min_year = data.index.min().year
    max_year = data.index.max().year

    grid = []
    for year in range(min_year, max_year+1):
        chunk = data[f'{year}0101':f'{year}1231']
        grid.append(chunk.groupby([dt.strftime(d, '%m') for d in chunk.index.to_pydatetime()]).max())
    
    if STEP == 5:
        pass

    fig.add_trace(
        go.Heatmap(
            x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], 
            y=list(range(min_year, max_year)),
            z=grid, colorscale='RdBu_r', showscale=False, showlegend=False),
    )


fig.write_image(f'weather_step{STEP + 1}.png')
fig

### Temperature shift

In [9]:
STEP = 2

mu = 13.
std = 1.8

if STEP == 0:
    fig = plot_extreme_pdf(mu, std, x_res=60, y_res=20)
elif STEP == 1:
    mu2 = 14.5
    fig = plot_extreme_pdf([mu, mu2], [std, std])
elif STEP == 2:
    mu2 = 14.5
    std2 = 2.5
    fig = plot_extreme_pdf([mu, mu2], [std, std2])

# general layout
fig.update_layout(
    width=800, height=450,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)

fig.write_image(f'tshift_step{STEP + 1}.png')
fig